# 蛋白质序列标签预测 - ProtVec 实现

本笔记本实现了使用 ProtVec（蛋白质特定嵌入）进行蛋白质序列标签预测的功能。ProtVec 是一种专门为蛋白质序列设计的嵌入方法，它通过将蛋白质序列分解为重叠的 n-gram（通常是 3-gram），然后使用 Word2Vec 类似的方法学习这些 n-gram 的向量表示。

In [1]:
import pickle
import gensim
import gensim.models

import os
import sys
import random
import numpy as np
import pandas as pd
from joblib import load, dump

from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import GaussianNB 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import classification_report

## 数据加载

首先，我们加载蛋白质序列数据。

In [2]:
# 加载数据
datas = pickle.load(open("../Data/WSAA_data_public.pkl", "rb"))

# 查看第一个样本
datas[0]

{'id': 'disordered_protein_0',
 'sequence': 'MKQFGLAAFDELKDGKYNDVNKTILEKQSVELRDQLMVFQERLVEFAKKHNSELQASPEFRSKFMHMCSSIGIDPLSLFDRDKHLFTVNDFYYEVCLKVIEICRQTKDMNGGVISFQELEKVHFRKLNVGLDDLEKSIDMLKSLECFEIFQIRGKKFLRSVPNELTSDQTKILEICSILGYSSISLLKANLGWEAVRSKSALDEMVANGLLWIDYQGGAEALYWDPSWITRQL',
 'label': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

## ProtVec 实现

ProtVec 是一种专门为蛋白质序列设计的嵌入方法，它通过将蛋白质序列分解为重叠的 n-gram（通常是 3-gram），然后使用 Word2Vec 类似的方法学习这些 n-gram 的向量表示。

下面我们实现 ProtVec 的训练过程：

In [3]:
def generate_ngrams(sequence, n=3):
    """生成蛋白质序列的n-gram"""
    return [sequence[i:i+n] for i in range(len(sequence)-n+1)]

# 为每个蛋白质序列生成n-gram
ngram_sentences = []
for data in datas:
    sequence = data["sequence"]
    ngrams = generate_ngrams(sequence, n=3)  # 使用3-gram
    ngram_sentences.append(ngrams)

# 设置随机种子以确保可重复性
random_seed = random.randint(0, 10000)

# 训练ProtVec模型
model_protvec = gensim.models.Word2Vec(
    sentences=ngram_sentences,
    vector_size=random.choice([10, 20, 40, 50, 100]),  # 随机选择向量维度
    window=5,  # 上下文窗口大小
    min_count=1,  # 最小词频
    workers=4,  # 并行训练的线程数
    seed=random_seed
)

# 查看向量维度
print(f"ProtVec向量维度: {model_protvec.vector_size}")

# 查看一些n-gram的向量表示
for ngram in ngram_sentences[0][:3]:
    print(f"n-gram: {ngram}, 向量: {model_protvec.wv[ngram][:5]}...")

ProtVec向量维度: 100
n-gram: MKQ, 向量: [-0.2966941   0.36993104 -0.43829548  0.26261833 -0.12605065]...
n-gram: KQF, 向量: [-0.18011813  0.3782929  -0.32417122  0.27704358 -0.16208789]...
n-gram: QFG, 向量: [-0.5383602  -0.1368353   0.19545464  0.08196804 -0.5159643 ]...


## 特征提取

现在，我们使用训练好的 ProtVec 模型为每个氨基酸提取特征。我们将考虑每个氨基酸周围的上下文（前后各2个氨基酸），并使用这些氨基酸所在的n-gram的平均向量作为特征。

In [4]:
def extract_features_for_position(sequence, position, model, n=3):
    """为序列中的特定位置提取特征"""
    # 获取包含该位置的所有n-gram
    relevant_ngrams = []
    for i in range(max(0, position-n+1), min(len(sequence)-n+1, position+1)):
        ngram = sequence[i:i+n]
        if ngram in model.wv:
            relevant_ngrams.append(ngram)
    
    # 如果没有相关n-gram，使用随机向量
    if not relevant_ngrams:
        return np.random.randn(model.vector_size)
    
    # 计算所有相关n-gram向量的平均值
    return np.mean([model.wv[ngram] for ngram in relevant_ngrams], axis=0)

# 提取特征
data_x = []
data_y = []

for data in datas:
    sequence = data["sequence"]
    labels = data["label"]
    
    for idx, label in enumerate(labels):
        # 提取该位置的特征
        features = extract_features_for_position(sequence, idx, model_protvec)
        data_x.append(features)
        data_y.append(label)

# 转换为numpy数组
data_x = np.array(data_x)
data_y = np.array(data_y)

# 查看数据形状
print(f"特征形状: {data_x.shape}")
print(f"标签形状: {data_y.shape}")

特征形状: (1266695, 100)
标签形状: (1266695,)


## 模型训练与评估

使用提取的特征训练分类模型，并评估其性能。

In [5]:
# 使用高斯朴素贝叶斯分类器
model = GaussianNB()

# 使用交叉验证进行预测
pred = cross_val_predict(
    model, data_x, data_y
)

# 打印分类报告
print(classification_report(data_y, pred))

              precision    recall  f1-score   support

           0       0.86      0.71      0.78   1043854
           1       0.25      0.45      0.32    222841

    accuracy                           0.66   1266695
   macro avg       0.55      0.58      0.55   1266695
weighted avg       0.75      0.66      0.70   1266695



## 尝试其他分类器

我们可以尝试其他分类器，看看是否能获得更好的性能。

In [6]:
# 使用逻辑回归分类器
model_lr = LogisticRegression(max_iter=1000, random_state=random_seed)

# 使用交叉验证进行预测
pred_lr = cross_val_predict(
    model_lr, data_x, data_y
)

# 打印分类报告
print("逻辑回归分类器性能：")
print(classification_report(data_y, pred_lr))

逻辑回归分类器性能：
              precision    recall  f1-score   support

           0       0.82      1.00      0.90   1043854
           1       0.48      0.01      0.01    222841

    accuracy                           0.82   1266695
   macro avg       0.65      0.50      0.46   1266695
weighted avg       0.76      0.82      0.75   1266695



## 保存模型

保存训练好的ProtVec模型和分类器，以便后续使用。

In [9]:
# 训练完整数据集上的模型
final_model = GaussianNB()
final_model.fit(data_x, data_y)

# 保存ProtVec模型
model_protvec.save("protvec_model.w2v")

# 保存分类器
dump(final_model, "classifier_model.joblib")

print("模型已保存！")

模型已保存！


## 结论

在这个笔记本中，我们实现了使用ProtVec（蛋白质特定嵌入）进行蛋白质序列标签预测的功能。ProtVec通过将蛋白质序列分解为重叠的n-gram，然后学习这些n-gram的向量表示，为蛋白质序列分析提供了有效的特征表示方法。

与基线模型中的Word2Vec相比，ProtVec更适合蛋白质序列分析，因为它考虑了蛋白质序列的特定结构和模式。通过使用n-gram捕获局部氨基酸模式，ProtVec能够更好地表示蛋白质序列中的功能和结构信息。